In [1]:
import random
from ipycanvas import Canvas

In [2]:
class Graph:
    def __init__(self, vertices=25, edges_to_remove=10):
        self.vertices = vertices
        self.width = int(vertices ** 0.5)
        self.structure = {i: [] for i in range(vertices)}
        self._form_structure()
        self._remove_edges(edges_to_remove)
        self._coords = {}
        self._calc_coords()
        
    def _form_structure(self):
        for i in range(self.vertices-1):
            if (i+1) % self.width != 0:
                self.structure[i].append(i+1)
                self.structure[i+1].append(i)
            if i < (self.vertices - self.width):
                self.structure[i].append(i+self.width)
                self.structure[i+self.width].append(i)
                
    def _calc_coords(self):
        for i in range(self.width):
            for t in range(self.width):
                self._coords[t+(i*5)] = [i, t]

    def _add_edge(self, v1, v2):
        self.structure[v1].append(v2)
        self.structure[v2].append(v1)

    def _delete_edge(self, v1, v2):
        self.structure[v1].remove(v2)
        self.structure[v2].remove(v1)

    def is_connected(self):
        not_visited = [i for i in range(self.vertices)]
        queue = [0]

        while queue:
            s = queue.pop(0)
            for neighbor in self.structure[s]:
                if neighbor in not_visited:
                    not_visited.remove(neighbor)
                    queue.append(neighbor)

        return not bool(not_visited)

    def _remove_edges(self, count):
        i = 0
        while i < count:
            vertex = random.randint(0, len(self.structure)-1)
            if len(self.structure[vertex])-1 == 0:
                continue
            neighbor = random.randint(0, len(self.structure[vertex])-1)
            neighbor = self.structure[vertex][neighbor]

            self._delete_edge(vertex, neighbor)
            if not self.is_connected():
                self._add_edge(vertex, neighbor)
            else:
                i += 1
                
    def draw(self, canv):
        coords = []
        for i in range(self.width):
            for t in range(self.width):
                coords.append([(i+1)*50,(t+1)*50])
                canv.fill_circle((i+1)*50, (t+1)*50, 8)
                canv.fill_style = "brown"
                canv.font = "16px sans-serif "
                canv.fill_text(t+(i*5), (i+1)*50+5, (t+1)*50-5)
                canv.fill_style = "black"
        for i in self.structure:
            for t in self.structure[i]:
                canv.stroke_line(coords[i][0], coords[i][1], coords[t][0], coords[t][1])
        return canv, coords  
        
    def get_neighs(self, vert):
        a = {}
        for i in self.structure[vert]:
            a[i] = self._coords[i]
        return a
    
    def get_coords(self, vertex):
        return self._coords[vertex]

In [3]:
def neighs_check(a, bs, c):
    x_ac = abs(c[0] - a[0])
    y_ac = abs(c[1] - a[1])
    closer, farther = set(), set()
    
    for key, value in bs:
        x_bc = abs(c[0] - value[0])
        y_bc = abs(c[1] - value[1])

        if (x_bc < x_ac) or (y_bc < y_ac):
            closer.add(key)
        else:
            farther.add(key)
            
    return closer, farther

In [4]:
def best_way(arr1, arr2):
    for i in arr1:
        for t in arr2:
            inter = i.intersection(t)
            if inter:
                return inter.pop()

In [5]:
class Agent():
    def __init__(self, pos, des, road):
        self.pos = pos
        self.des = des
        self.road = road
        self.neighs = road.get_neighs(pos)
        self.map = {pos: []}
        self.hist = []

    def _step(self, vert):
        if vert not in self.map[self.pos]:
            self.map[self.pos].append(vert)
            
        self.hist.append([self.pos, vert])
        self.pos = vert
        self.neighs = self.road.get_neighs(self.pos)
        
        if self.pos not in self.map:
            self.map[self.pos] = []

    def _choose_way(self):
        if sorted(self.neighs.keys()) == sorted(self.map[self.pos]):
            self.map[self.pos] = []
            
        pr1, pr2 = set(), set()
        
        for i in self.neighs.keys():
            if i not in self.map.keys():
                pr1.add(i)
            elif i not in self.map[self.pos]:
                pr2.add(i)
                
        a = self.road.get_coords(self.pos)
        c = self.road.get_coords(self.des)
        cl, fr = neighs_check(a, self.neighs.items(), c)
        
        return best_way([pr1, pr2], [cl, fr])

    def move(self):
        while self.pos != self.des:
            vert = self._choose_way()
            self._step(vert)

    def draw(self, canv, coords):
        canv.fill_style = "orange"
        for i in self.map:
            for t in self.map[i]:
                canv.stroke_line(coords[i][0], coords[i][1], coords[t][0], coords[t][1])
            x, y = coords[i]
            canv.fill_circle(x, y, 8)
            canv.fill_style = "brown"
            canv.font = "16px sans-serif "
            canv.fill_text(i, x+5, y-5)
            canv.fill_style = "black"
        canv.fill_style = "blue"
        x, y = coords[self.pos]
        canv.fill_circle(x, y, 8)
        canv.fill_style = "black"
        return canv

In [6]:
graph = Graph()
# print(graph.structure)
canvas1 = Canvas(width=300, height=300)
canvas1, crds = graph.draw(canvas1)
canvas1

Canvas(height=300, width=300)

In [10]:
# a = random.randint(0, 24)
# b = random.randint(0, 24)
a = 0
b = 24

agent = Agent(a, b, graph)
agent.move()

print('Initial vertex:', a, '(orange)')
print('Final vertex:', b, '(blue)')
print('Steps:', len(agent.hist))
print('Way:')
for i in agent.hist:
    print(i[0], '->', end=' ')
print(agent.hist[len(agent.hist)-1][1])

canvas2 = Canvas(width=300, height=300)
agent.draw(canvas2, crds)

Initial vertex: 18 (orange)
Final vertex: 6 (blue)
Steps: 4
Way:
18 -> 17 -> 16 -> 11 -> 6


Canvas(height=300, width=300)